In [ ]:
import pandas as pd

In [ ]:
def compare_dataframes(df_actual, df_expected):
    if not df_actual.reset_index(drop=True).equals(df_expected.reset_index(drop=True)):
        raise RuntimeError(f'\nОжидаемое значение:\n {df_expected} \nФактическое:\n {df_actual}')

def compare_series(ser_actual, ser_expected):
    if not ser_actual.equals(ser_expected):
        raise RuntimeError(f'\nОжидаемое значение:\n {ser_expected} \nФактическое:\n {ser_actual}')

def calculate_lines_of_code(f):
    import inspect

    counting = True
    counter = 0

    for line in inspect.getsourcelines(f)[0]:
        if line.strip() == '"""':
            counting = not counting
            continue

        if line.strip() == '':
            continue

        if counting:
            counter += 1

    return counter - 1

## Цепочки методов

In [ ]:
def modify_columns_solution(df, column_to_drop, column_to_change):
    """
    Преобразует заданную таблицу типа DataFrame с помощью цепочки методов.

    Аргументы:
        df: исходная таблица.
        column_to_drop: название колонки, которую нужно удалить.
        column_to_change: название колонки, которую нужно изменить.

    Возвращаемое значение:
        таблица, преобразованная с помощью цепочки методов:
            1. Добавление в исходную таблицу новой колонки с названием new,
               которая получена на основе колонки column_to_change с помощью прибавления 1.
            2. Удаление колонки column_to_drop.
            3. Переименование колонки column_to_change в колонку с названием renamed.
    """
    return df.assign(new=lambda x: x[column_to_change] + 1).drop(column_to_drop, axis=1, inplace=False).rename(columns=lambda x: 'renamed' if x == column_to_change else x)

In [ ]:
def _test_modify_columns(df, column_to_drop, column_to_change, expected_df):
    compare_dataframes(modify_columns_solution(df, column_to_drop, column_to_change), expected_df)

def test_modify_columns_1():
    df = pd.DataFrame({
        'column1': [1, 2, 3, 4],
        'column2': [5, 6, 7, 8],
        'column3': [None, None, None, None]
    })
    column_to_drop = 'column3'
    column_to_change = 'column1'
    expected_df = pd.DataFrame({
        'renamed': [1, 2, 3, 4],
        'column2': [5, 6, 7, 8],
        'new': [2, 3, 4, 5]
    })
    _test_modify_columns(df, column_to_drop, column_to_change, expected_df)

def test_modify_columns_2():
    df = pd.DataFrame({
        'column1': [1, 2, 3, 4]
    })
    column_to_drop = 'column1'
    column_to_change = 'column1'
    expected_df = pd.DataFrame({
        'new': [2, 3, 4, 5]
    })
    _test_modify_columns(df, column_to_drop, column_to_change, expected_df)

def modify_columns_test():
    if calculate_lines_of_code(modify_columns_solution) != 1:
        raise RuntimeError('В теле функции должна быть ровно одна строка!')

    test_modify_columns_1()
    test_modify_columns_2()

    print('Все тесты прошли успешно!')

In [ ]:
modify_columns_test()

Все тесты прошли успешно!


In [ ]:
def filter_group_aggregate_sort_solution(df, column_to_filter, filter_set, column_to_group, column_to_aggregate):
    """
    Преобразует заданную таблицу типа DataFrame с помощью цепочки методов.

    Аргументы:
        df: исходная таблица.
        column_to_filter: название колонки, по которой нужно провести фильтрацию.
        filter_set: множество значений, согласно которому нужно провести фильтрацию.
        column_to_group: название колонки, по которой нужно провести группировку.
        column_to_aggregate: название колонки, по которой нужно провести агрегацию.

    Возвращаемое значение:
        таблица, преобразованная с помощью цепочки методов:
            1. Фильтрация исходной таблицы по следующему принципу: необходимо оставить строки,
               для которых значение в колонке column_to_filter содержится во множестве filter_set.
            2. Группировка строчек в таблице по значениям в колонке column_to_group
               (группировку нужно проводить со значением аргумента as_index=False).
            3. Агрегация по колонке column_to_aggregate с функцией агрегации sum.
            4. Сортировка строчек в таблице в порядке убывания значений
               в колонке column_to_aggregate.
    """
    return df[df[column_to_filter].isin(filter_set)].groupby(column_to_group, as_index=False).agg({column_to_aggregate: 'sum'}).sort_values(column_to_aggregate, ascending=False)

In [ ]:
def _test_filter_group_aggregate_sort(df, column_to_filter, filter_set, column_to_group, column_to_aggregate, expected_df):
    compare_dataframes(filter_group_aggregate_sort_solution(df, column_to_filter, filter_set, column_to_group, column_to_aggregate), expected_df)

def test_filter_group_aggregate_sort_1():
    df = pd.DataFrame({
        'column1': ['a', 'a', 'b', 'b'],
        'column2': [1, 2, 3, 4],
        'column3': [10, -1, 6, 8]
    })
    column_to_filter = 'column3'
    filter_set = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    column_to_group = 'column1'
    column_to_aggregate = 'column2'
    expected_df = pd.DataFrame({
        'column1': ['b', 'a'],
        'column2': [7, 1]
    })
    _test_filter_group_aggregate_sort(df, column_to_filter, filter_set, column_to_group, column_to_aggregate, expected_df)

def test_filter_group_aggregate_sort_2():
    df = pd.DataFrame({
        'column1': [1, 2, 3, 4, 1]
    })
    column_to_filter = 'column1'
    filter_set = {1, 4}
    column_to_group = 'column1'
    column_to_aggregate = 'column1'
    expected_df = pd.DataFrame({
        'column1': [4, 2]
    })
    _test_filter_group_aggregate_sort(df, column_to_filter, filter_set, column_to_group, column_to_aggregate, expected_df)

def filter_group_aggregate_sort_test():
    if calculate_lines_of_code(filter_group_aggregate_sort_solution) != 1:
        raise RuntimeError('В теле функции должна быть ровно одна строка!')

    test_filter_group_aggregate_sort_1()
    test_filter_group_aggregate_sort_2()

    print('Все тесты прошли успешно!')

In [ ]:
filter_group_aggregate_sort_test()

Все тесты прошли успешно!


## Полезные функции и методы

In [ ]:
def pivot_table_solution(df, index_column, columns_column, values_column):
    """
    Преобразует заданную таблицу типа `DataFrame` в сводную таблицу.

    Аргументы:
        df: исходная таблица.
        index_column: название колонки, значения из которой должны стать индексом в сводной таблице.
        columns_column: название колонки, значения из которой должны стать колонками в сводной таблице;
        values_column: название колонки, значения из которой должны стать колонками в сводной таблице.

    Возвращаемое значение:
        сводная таблица, в которой строчки и столбцы упорядочены по возрастанию индекса.
    """
    return df.pivot(index=index_column, columns=columns_column, values=values_column).sort_index(axis=0, ascending=True).sort_index(axis=1, ascending=True)

In [ ]:
def _test_pivot_table(df, index_column, columns_column, values_column, expected_df):
    compare_dataframes(pivot_table_solution(df, index_column, columns_column, values_column), expected_df)

def test_pivot_table_1():
    df = pd.DataFrame({
        'subject': ['Биология', 'Химия', 'Математика', 'Математика', 'Химия', 'Математика', 'Математика', 'Химия', 'Биология'],
        'grade': [9, 8, 5, 7, 9, 6, 10, 10, 8],
        'count': [183, 109, 534, 340, 129, 402, 173, 144, 197]
    })
    index_column = 'subject'
    columns_column = 'grade'
    values_column = 'count'
    expected_df = pd.DataFrame([
            [None, None, None, 197, 183, None],
            [534, 402, 340, None, None, 173],
            [None, None, None, 109, 129, 144]
        ],
        index=['Биология', 'Математика', 'Химия'],
        columns=[5, 6, 7, 8, 9, 10]
    )
    expected_df.index.name = 'subject'
    expected_df.columns.name = 'grade'
    _test_pivot_table(df, index_column, columns_column, values_column, expected_df)

def pivot_table_test():
    if calculate_lines_of_code(pivot_table_solution) != 1:
        raise RuntimeError('В теле функции должна быть ровно одна строка!')

    test_pivot_table_1()

    print('Все тесты прошли успешно!')

In [ ]:
pivot_table_test()

Все тесты прошли успешно!


In [ ]:
def count_courses_solution(df):
    """
    Преобразует заданную таблицу типа DataFrame с помощью цепочки методов.

    Аргументы:
        df: исходная таблица.

    Возвращаемое значение:
        таблица, преобразованная с помощью цепочки методов:
            1. Изменение колонки courses: разбиение значений в колонке
               с помощью функции split по разделителю «, ».
            2. Приведение с помощью функции explode таблицы к виду, когда в каждой
               строке в колонке courses записано ровно одно значение идентификатора курса.
            3. Подсчёт с помощью функции value_counts частоты встречаемости курсов в колонке courses.

    """
    return df['courses'].str.split(', ').explode().value_counts()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
base = 'drive/MyDrive/data'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def _test_count_courses(ser, expected_ser):
    compare_series(ser, expected_ser)

def test_count_courses_1():
    df = pd.read_csv(base+'/courses_test1.csv')
    expected_ser = pd.Series([12, 12, 11, 8])
    expected_ser.index = ['1235', '1236', '1237', '1234']
    expected_ser.index.name = 'courses'
    _test_count_courses(count_courses_solution(df), expected_ser)

def count_courses_test():
    if calculate_lines_of_code(count_courses_solution) != 1:
        raise RuntimeError('В теле функции должна быть ровно одна строка!')

    test_count_courses_1()

    print('Все тесты прошли успешно!')

In [ ]:
count_courses_test()

Все тесты прошли успешно!


## Заготовки для вычислительных задач

В файле `user.csv` (<a class="ui-file-link" href="https://edu.sirius.online/noo-back/files/ac515a3d068d43bcd3fb4c9b98cba21970ae6b83.csv" download="user.csv">user.csv</a>) содержится таблица `user` с полями `id`, `grade` и `region`.

С помощью функции `sample` сделайте случайную выборку из этой таблицы, содержащую $100$ строк. В качестве зерна (параметр `random_state`) используйте число $42$. Вычислите сумму значений в колонке `grade` получившейся выборки.

In [ ]:
user_df = pd.read_csv(base + '/user.csv')

In [ ]:
user_df.sample(n=100, random_state=42)['grade'].sum()

756.0

В файле `temperature.csv` (<a class="ui-file-link" href="https://edu.sirius.online/noo-back/files/75dfda35914787f4e8dff30a1e01e020b25dea1e.csv" download="temperature.csv">temperature.csv</a>) содержится таблица с единственным полем `temperature` — значением температуры. У таблицы имеется отсортированный по возрастанию индекс, содержащий ежеминутные моменты времени. Температура записывается только в том случае, если она изменилась по сравнению с предыдущим моментом времени, поэтому в таблице присутствуют пропуски.

Заполните пропуски в данных с помощью функции `ffill`. Вычислите среднюю температуру за весь период с учётом заполненных пропусков. Ответ округлите до двух знаков после запятой.

In [ ]:
temperature_df = pd.read_csv(base+ '/temperature.csv', index_col=0)

In [ ]:
temperature_df = temperature_df.ffill()
temperature_df.mean()

,0
temperature,19.767152


В файле `courses.csv` (<a class="ui-file-link" href="https://edu.sirius.online/noo-back/files/3a181a08998d025dc24ea6d0a0bca312b53bf34d.csv" download="courses.csv">courses.csv</a>) содержится таблица с двумя колонками:
1. `id` — идентификатор ученика;
2. `courses` — список идентификаторов курсов, на которые записан ученик.

Значения в колонке `courses` представляют собой строки, содержащие идентификаторы курсов, разделённые запятой с пробелом («`, `»). Гарантируется, что значения в поле `id` уникальны.

Последовательно применяя функции `split`, `explode` и `value_counts`, определите, на какой курс записано больше всего учеников.


In [ ]:
courses = pd.read_csv(base +  '/courses.csv')

In [ ]:
courses
courses['courses'] = courses['courses'].str.split(', ')
courses
courses = courses.explode('courses')
courses
courses.groupby('courses').agg({'id': 'count'}).sort_values(by='id', ascending=False)

,id
courses,
1240,83
1241,71
1238,64
1242,64
1236,63
1234,61
1236,61
1239,57
1241,55
